In [2]:
! pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.6 MB/s eta 0:00:00


In [3]:
! pip install Crypto

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 kB 9.4 MB/s eta 0:00:00


In [4]:
# import the necessary libraries here
import Crypto
from Crypto.Random import random
import Crypto.Random
import Crypto.Util
from Crypto.Util.number import bytes_to_long, long_to_bytes
import random

import Crypto.Util.number


def modular_inv(a,b):
    x = 1
    y = 0
    x1 = 0
    y1 = 1
    a1 = a
    b1 = b

    while b1!=0:
        q = a1//b1
        (x,x1) = (x1, x - q*x1)
        (y,y1) = (y1, y - q*y1)
        (a1,b1) = (b1, a1 - q*b1)
    return x,y


class RSA:
    """Implements the RSA public key encryption / decryption."""

    def __init__(self, key_length):
        # generate public and private keys here
        self.p = Crypto.Util.number.getPrime(key_length)
        self.q = Crypto.Util.number.getPrime(key_length)
        self.n = self.p * self.q
        self.phi = (self.p - 1) * (self.q - 1)

        extra_bits = random.randint(1,key_length-1)
        self.e = Crypto.Util.number.getPrime(key_length+extra_bits)
        # If e has more bits than p,q and e is prime then it is coprime with (p-1)(q-1)
        self.d = (modular_inv( self.phi , self.e)[1]) % self.phi


    def encrypt(self, binary_data):
        # return encryption of binary_data here
        return pow(bytes_to_long(binary_data), self.e, self.n)


    def decrypt(self, encrypted_int_data):
        # return decryption of encrypted_binary_data here
        return long_to_bytes(pow(encrypted_int_data, self.d, self.n)).decode()


class RSAParityOracle(RSA):
    """Extends the RSA class by adding a method to verify the parity of data."""

    def is_parity_odd(self, encrypted_int_data):
        # Decrypt the input data and return whether the resulting number is odd
        if pow(encrypted_int_data, self.d, self.n) % 2 == 1:
          return True
        else:
          return False


def parity_oracle_attack(ciphertext, rsa_parity_oracle):
    left = 0
    right = rsa_parity_oracle.n - 1
    power = pow(2,rsa_parity_oracle.e,rsa_parity_oracle.n)
    original_text = ciphertext
    while left < right:
        mid = (left+right)//2
        ciphertext = (ciphertext*power)%rsa_parity_oracle.n
        if rsa_parity_oracle.is_parity_odd(ciphertext):
            left = mid + 1
        else:
            right = mid
    left = left & (~0xff)
    for i in range(0,256):
        try:
            if (rsa_parity_oracle.encrypt(long_to_bytes(left+i)) == original_text): # Encryption key is visible to all so
                return long_to_bytes(left+i)
        except UnicodeDecodeError:
            pass



def main():
    input_bytes = input("Enter the message: ")

    # Generate a 1024-bit RSA pair
    rsa_parity_oracle = RSAParityOracle(1024)

    # Encrypt the message
    ciphertext = rsa_parity_oracle.encrypt(input_bytes.encode())
    print("Encrypted message is: ",ciphertext)
    # print("Decrypted text is: ",rsa_parity_oracle.decrypt(ciphertext))

    # Check if the attack works
    plaintext = parity_oracle_attack(ciphertext, rsa_parity_oracle)
    print("Obtained plaintext: ",plaintext)
    assert plaintext == input_bytes.encode()


if __name__ == '__main__':
    main()

Enter the message: jigyasachouhan
Encrypted message is:  10577724835682171644832978333803918189287884891569085384279987352894801411312383388813928764019502771731395590398400457716381716341552352193830367706894469317542833079258110836859970443184482871466866820633783634931001474178402373117667617620151784536430833461695719980290924337671004276820028695088572199809481025371142230970567083649702752915954750089349175017346436238272773028875912649527608003623575118180932237321980778931028243760747761298903057419133566412097209071359452029216250960976542088848889346869823750752022694305095150714435454350537714546934358574240388098169644239558236541065078412656114079796249
Obtained plaintext:  b'jigyasachouhan'
